In [44]:
import os
import re

N_values = set()
D_values = set()
K_values = set()

latency_values = {}
tps_values = {}

get_key = lambda indexed, N, D, K: f"{'true' if indexed else 'false'}_N{N}_D{D}_K{K}"

dir = 'outputs'
file_names = os.listdir(dir)

for file_name in file_names:
    key = file_name.split('.')[0]
    parts = key.split('_')
    if len(parts) != 4:
        continue
    indexed = parts[0]
    N_values.add(parts[1][1:])
    D_values.add(parts[2][1:])
    K_values.add(parts[3][1:])

    # Read the file and extract the latency average value
    with open(f"{dir}/{file_name}", 'r') as file:
        file_content = file.read()

        # Extract latency average using regular expression
        latency_average_match = re.search(r'latency average = (\d+\.\d+) ms', file_content)
        if latency_average_match:
            latency_values[key] = float(latency_average_match.group(1))

        # Extract TPS (Transactions Per Second) using regular expression
        tps_match = re.search(r'tps = (\d+\.\d+)', file_content)
        if tps_match:
            tps_values[key] = float(tps_match.group(1))

N_values = sorted([int(N) for N in N_values])
D_values = sorted([int(D) for D in D_values])
K_values = sorted([int(K) for K in K_values])

# 10 Shades of Green from lighter to darker
green_shades = [
    'rgb(153,255,153)', 
    'rgb(102,255,102)',  
    'rgb(51,255,51)',  
    'rgb(0,255,0)',  # Lime
    'rgb(0,204,0)',  
    'rgb(0,153,0)',  
]

# 10 Shades of Red from lighter to darker
red_shades = [
    'rgb(255,153,153)',  
    'rgb(255,102,102)',  
    'rgb(255,51,51)',  
    'rgb(255,0,0)',  # Red
    'rgb(204,0,0)',  
    'rgb(153,0,0)',
]

In [46]:
import plotly.graph_objects as go

full_strings = {
    'N': 'Number of rows (N)',
    'D': 'Dimensions of vectors (D)',
    'K': 'Number of similar vectors (K)'
}

param_values = {
    'D': D_values,
    'N': N_values,
    'K': K_values
}

def generate_plot(x, y, key_to_values_dict, per, fixed, fixed_value):
    # Process data
    plot_items = []
    indexed_count = 0
    unindexed_count = 0
    for indexed in [True, False]:
        for param_per in param_values[per]:
            x_values = []
            y_values = []
            for param_x in param_values[x]:
                key_params = {}
                key_params[x] = param_x
                key_params[per] = param_per
                key_params[fixed] = fixed_value

                key = get_key(indexed, key_params['N'], key_params['D'], key_params['K'])
                if key in key_to_values_dict:
                    x_values.append(param_x)
                    y_values.append(key_to_values_dict[key])
            if len(x_values) > 0:
                if indexed:
                    indexed_str = 'indexed'
                    color = green_shades[indexed_count]
                    indexed_count += 1
                else:
                    indexed_str = 'unindexed'
                    color = red_shades[unindexed_count]
                    unindexed_count += 1
                plot_items.append((f"{param_per} ({indexed_str})", x_values, y_values, color))

    # Plot data
    fig = go.Figure()
    for (key, x_values, y_values, color) in plot_items:
        fig.add_trace(go.Scatter(
            x=x_values,
            y=y_values,
            marker=dict(color=color),
            mode='lines+markers',
            name=key
        ))
    fig.update_layout(
        title=f"{y} vs. {x} per {per}",
        xaxis_title=full_strings[x],
        yaxis_title=y
    )
    fig.show()


generate_plot(x='N', y='Latency (ms)', key_to_values_dict=latency_values, per='D', fixed='K', fixed_value=4)
generate_plot(x='K', y='Latency (ms)', key_to_values_dict=latency_values, per='D', fixed='N', fixed_value=100000)
generate_plot(x='D', y='Latency (ms)', key_to_values_dict=latency_values, per='N', fixed='K', fixed_value=4)
generate_plot(x='N', y='TPS', key_to_values_dict=tps_values, per='D', fixed='K', fixed_value=4)
generate_plot(x='K', y='TPS', key_to_values_dict=tps_values, per='D', fixed='N', fixed_value=100000)
generate_plot(x='D', y='TPS', key_to_values_dict=tps_values, per='N', fixed='K', fixed_value=4)